# Progressive Rollouts with Single Seldon Deployment

In this example we will AB Test two Iris models: an SKLearn model and an XGBOOST model.
We will run a progressive rollout allowing Iter8 to control the traffic to the two Seldon Deployments and gradually move traffic to the best model.

## Install Depenendcies

  * Istio
  * Seldon Core
  * Seldon Core Analytics
  * Iter8
 
  
  You can create a Kind cluster with all dependencies installed with [Ansible](https://www.ansible.com/) with:
  
  ```
  pip install ansible openshift
  ansible-galaxy collection install git+https://github.com/SeldonIO/ansible-k8s-collection.git,v0.1.0
  ```
  
  Then from `example/iter8` folder run:
  
  ```
  ansible-playbook playbooks/iter8.yml
  ```

## Create ABTest with Two Predictors

In [1]:
!cat abtest.yaml

apiVersion: v1
kind: Namespace
metadata:
    name: ns-production
---
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: iris
  namespace: ns-production
spec:
  predictors:
  - name: baseline
    traffic: 100    
    graph:
      name: classifier
      modelUri: gs://seldon-models/v1.14.0-dev/sklearn/iris
      implementation: SKLEARN_SERVER
  - name: candidate
    traffic: 0
    graph:
      name: classifier
      modelUri: gs://seldon-models/xgboost/iris
      implementation: XGBOOST_SERVER



In [2]:
!kubectl apply -f abtest.yaml

namespace/ns-production created
seldondeployment.machinelearning.seldon.io/iris created


In [3]:
!kubectl wait --for condition=ready --timeout=600s pods --all -n ns-production

pod/iris-baseline-0-classifier-5759fd6c8-j4hc2 condition met
pod/iris-candidate-0-classifier-6d6d54786c-nthhm condition met


## Create some load on models.

We will send reqeusts which will be split by the Seldon AB Test as well as random feedback to both models with feedback favouring the candidate

In [4]:
!cat fortio.yaml

---
apiVersion: batch/v1
kind: Job
metadata:
  name: fortio-requests
  namespace: default
spec:
  template:
    spec:
      volumes:
      - name: shared
        emptyDir: {}    
      containers:
      - name: fortio
        image: fortio/fortio
        command: [ 'fortio', 'load', '-t', '6000s', '-qps', "5", '-json', '/shared/fortiooutput.json', '-H', 'Content-Type: application/json', '-payload', '{"data": {"ndarray":[[6.8,2.8,4.8,1.4]]}}',  "$(URL)" ]
        env:
        - name: URL
          value: URL_VALUE/seldon/ns-production/iris/api/v1.0/predictions
        volumeMounts:
        - name: shared
          mountPath: /shared         
      - name: busybox
        image: busybox:1.28
        command: ['sh', '-c', 'echo busybox is running! && sleep 6000']          
        volumeMounts:
        - name: shared
          mountPath: /shared       
      restartPolicy: Never
---
apiVersion: batch/v1
kind: Job
metadata:
  name: fortio-irisv1-rewards
  namespace: default  
spec:
  templ

In [1]:
!URL_VALUE="http://$(kubectl -n istio-system get svc istio-ingressgateway -o jsonpath='{.spec.clusterIP}')" && \
  sed "s+URL_VALUE+${URL_VALUE}+g" fortio.yaml | kubectl apply -f -

job.batch/fortio-requests created
job.batch/fortio-irisv1-rewards created
job.batch/fortio-irisv2-rewards created


In [2]:
!kubectl wait --for condition=ready --timeout=600s pods --all -n default

pod/fortio-irisv1-rewards-5srd4 condition met
pod/fortio-irisv2-rewards-jzs4s condition met
pod/fortio-requests-lrzhp condition met


## Create Metrics to evaluate 

These are a standard set of metrics we use in all examples.

In [7]:
!cat ../../metrics.yaml

apiVersion: v1
kind: Namespace
metadata:
  name: iter8-seldon
---
apiVersion: iter8.tools/v2alpha2
kind: Metric
metadata:
  name: 95th-percentile-tail-latency
  namespace: iter8-seldon
spec:
  description: 95th percentile tail latency
  jqExpression: .data.result[0].value[1] | tonumber
  params:
  - name: query
    value: |
      histogram_quantile(0.95, sum(rate(seldon_api_executor_client_requests_seconds_bucket{seldon_deployment_id='$sid',predictor_name='$predictor',kubernetes_namespace='$ns'}[${elapsedTime}s])) by (le))
  provider: prometheus
  sampleSize: iter8-seldon/request-count
  type: Gauge
  units: milliseconds
  urlTemplate: http://seldon-core-analytics-prometheus-seldon.seldon-system/api/v1/query
---
apiVersion: iter8.tools/v2alpha2
kind: Metric
metadata:
  name: error-count
  namespace: iter8-seldon
spec:
  description: Number of error responses
  jqExpression: .data.result[0].value[1] | tonumber
  params:
  - name: query
    value: |
      sum(increase(seldon_api_executor

In [8]:
!kubectl create -f ../../metrics.yaml

namespace/iter8-seldon created
metric.iter8.tools/95th-percentile-tail-latency created
metric.iter8.tools/error-count created
metric.iter8.tools/error-rate created
metric.iter8.tools/mean-latency created
metric.iter8.tools/request-count created
metric.iter8.tools/user-engagement created


In [9]:
!kubectl get metrics -n iter8-seldon

NAME                           TYPE      DESCRIPTION
95th-percentile-tail-latency   Gauge     95th percentile tail latency
error-count                    Counter   Number of error responses
error-rate                     Gauge     Fraction of requests with error responses
mean-latency                   Gauge     Mean latency
request-count                  Counter   Number of requests
user-engagement                Gauge     Number of feedback requests


## Create Progressive Rollout Experiment

  * Run 15 iterations with 5 second gaps between default and candidate models
  * Both models must pass objectives
  * winnder will be chosen based on user engagement metric

In [3]:
!cat experiment.yaml

apiVersion: iter8.tools/v2alpha2
kind: Experiment
metadata:
  name: quickstart-exp
spec:
  target: iris
  strategy:
    testingPattern: A/B
    deploymentPattern: Progressive
    actions:
      # when the experiment completes, promote the winning version using kubectl apply
      finish:
      - task: common/exec
        with:
          cmd: /bin/bash
          args: [ "-c", "kubectl apply -f {{ .promote }}" ]
  criteria:
    requestCount: iter8-seldon/request-count
    rewards: # Business rewards
    - metric: iter8-seldon/user-engagement
      preferredDirection: High # maximize user engagement
    objectives:
    - metric: iter8-seldon/mean-latency
      upperLimit: 2000
    - metric: iter8-seldon/95th-percentile-tail-latency
      upperLimit: 5000
    - metric: iter8-seldon/error-rate
      upperLimit: "0.01"
  duration:
    intervalSeconds: 10
    iterationsPerLoop: 15
  versionInfo:
    # information about model versions used in this experiment
    baseline:
      name: iris-v1
 

In [4]:
!kubectl create -f experiment.yaml

experiment.iter8.tools/quickstart-exp created


## Monitor Experiment

Download iter8ctl. 

```
GO111MODULE=on GOBIN=/usr/local/bin go get github.com/iter8-tools/iter8ctl@v0.1.3
```

Then:

```
while clear; do kubectl get experiment quickstart-exp -o yaml | iter8ctl describe -f -; sleep 8; done
```

By the end you should see the xgboost candidate model is promoted.

In [5]:
!kubectl wait experiment quickstart-exp --for=condition=Completed --timeout=300s

experiment.iter8.tools/quickstart-exp condition met


In [6]:
!kubectl get experiment quickstart-exp -o yaml

apiVersion: iter8.tools/v2alpha2
kind: Experiment
metadata:
  creationTimestamp: "2021-06-15T15:20:18Z"
  generation: 1
  managedFields:
  - apiVersion: iter8.tools/v2alpha2
    fieldsType: FieldsV1
    fieldsV1:
      f:spec:
        .: {}
        f:criteria:
          .: {}
          f:requestCount: {}
          f:rewards: {}
        f:duration:
          .: {}
          f:intervalSeconds: {}
          f:iterationsPerLoop: {}
        f:strategy:
          .: {}
          f:actions:
            .: {}
            f:finish: {}
          f:deploymentPattern: {}
          f:testingPattern: {}
        f:target: {}
        f:versionInfo:
          .: {}
          f:baseline:
            .: {}
            f:name: {}
            f:variables: {}
            f:weightObjRef:
              .: {}
              f:apiVersion: {}
              f:fieldPath: {}
              f:kind: {}
              f:name: {}
              f:namespace: {}
          f:candidates: {}
    manager: kubectl-create
    oper

## Cleanup

In [7]:
!kubectl delete -f fortio.yaml
!kubectl delete -f experiment.yaml
!kubectl delete -f ../../metrics.yaml
!kubectl delete -f abtest.yaml

job.batch "fortio-requests" deleted
job.batch "fortio-irisv1-rewards" deleted
job.batch "fortio-irisv2-rewards" deleted
experiment.iter8.tools "quickstart-exp" deleted
namespace "iter8-seldon" deleted
metric.iter8.tools "95th-percentile-tail-latency" deleted
metric.iter8.tools "error-count" deleted
metric.iter8.tools "error-rate" deleted
metric.iter8.tools "mean-latency" deleted
metric.iter8.tools "request-count" deleted
metric.iter8.tools "user-engagement" deleted
namespace "ns-production" deleted
seldondeployment.machinelearning.seldon.io "iris" deleted
